# **موتور جستجو با تحلیل پیشرفته**

In [4]:
# نصب کتابخانه‌های مورد نیاز
!pip install -q faiss-cpu torch torchvision transformers gradio pillow requests tqdm matplotlib seaborn scikit-learn

import os
import torch
import numpy as np
import faiss
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from tqdm import tqdm
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# Import models
from torchvision import models, transforms
from transformers import CLIPModel, CLIPProcessor, ViTModel, ViTImageProcessor

class MultiModalImageSearchEngine:
    """موتور جستجوی تصویری چند مدله"""

    def __init__(self, model_types=['resnet50', 'vit', 'clip']):
        self.model_types = model_types
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = {}
        self.processors = {}
        self.transforms = {}
        self.feature_dims = {}
        self.indices = {}
        self.image_paths = []
        self.features = {}

        print(f"🚀 بارگذاری مدل‌ها روی {self.device}...")
        self._load_all_models()

    def _load_all_models(self):
        """بارگذاری همه مدل‌ها"""

        # 1. ResNet50 (CNN)
        if 'resnet50' in self.model_types:
            print("📦 بارگذاری ResNet50...")
            self.models['resnet50'] = models.resnet50(pretrained=True)
            self.models['resnet50'] = torch.nn.Sequential(*list(self.models['resnet50'].children())[:-1])
            self.models['resnet50'].to(self.device)
            self.models['resnet50'].eval()
            self.feature_dims['resnet50'] = 2048

            self.transforms['resnet50'] = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
            ])

        # 2. Vision Transformer (ViT)
        if 'vit' in self.model_types:
            print("📦 بارگذاری ViT...")
            self.processors['vit'] = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
            self.models['vit'] = ViTModel.from_pretrained('google/vit-base-patch16-224')
            self.models['vit'].to(self.device)
            self.models['vit'].eval()
            self.feature_dims['vit'] = 768

        # 3. CLIP
        if 'clip' in self.model_types:
            print("📦 بارگذاری CLIP...")
            self.models['clip'] = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
            self.processors['clip'] = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            self.models['clip'].to(self.device)
            self.models['clip'].eval()
            self.feature_dims['clip'] = 512

        print("✅ همه مدل‌ها بارگذاری شدند!")

    def extract_features(self, image, model_type):
        """استخراج ویژگی با مدل مشخص"""
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')

        with torch.no_grad():
            if model_type == 'resnet50':
                img_tensor = self.transforms['resnet50'](image).unsqueeze(0).to(self.device)
                features = self.models['resnet50'](img_tensor).squeeze().cpu().numpy()

            elif model_type == 'vit':
                inputs = self.processors['vit'](images=image, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                outputs = self.models['vit'](**inputs)
                features = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

            elif model_type == 'clip':
                inputs = self.processors['clip'](images=image, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                features = self.models['clip'].get_image_features(**inputs).squeeze().cpu().numpy()

        return features / np.linalg.norm(features) # نرمال‌سازی

    def add_similarity_text_to_image(self, image, similarity_score, rank=None):
        """اضافه کردن متن شباهت روی تصویر"""
        img_copy = image.copy()
        draw = ImageDraw.Draw(img_copy)

        try:
            font = ImageFont.load_default()
        except:
            font = ImageFont.load_default()

        if rank:
            text = f"#{rank} - {similarity_score:.1%}"
        else:
            text = f"{similarity_score:.1%}"

        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        x = img_copy.width - text_width - 10
        y = 10

        padding = 5
        draw.rectangle([
            x - padding,
            y - padding,
            x + text_width + padding,
            y + text_height + padding
        ], fill=(0, 0, 0, 180))

        draw.text((x, y), text, fill=(255, 255, 255), font=font)

        return img_copy

    def create_similarity_chart(self, results, query_type="تصویری"):
        """ایجاد نمودار شباهت"""
        if not results:
            return None

        # تنظیمات فارسی
        plt.rcParams['font.family'] = ['DejaVu Sans']

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # نمودار میله‌ای شباهت
        ranks = [f"#{r['rank']}" for r in results[:10]]
        similarities = [r['similarity'] * 100 for r in results[:10]]

        colors = plt.cm.viridis(np.linspace(0, 1, len(similarities)))
        bars = ax1.bar(ranks, similarities, color=colors)

        ax1.set_title(f'📊 میزان شباهت - جستجوی {query_type}', fontsize=14, fontweight='bold')
        ax1.set_xlabel('رتبه تصاویر', fontsize=12)
        ax1.set_ylabel('درصد شباهت', fontsize=12)
        ax1.set_ylim(0, 100)

        # اضافه کردن مقادیر روی میله‌ها
        for bar, sim in zip(bars, similarities):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{sim:.1f}%', ha='center', va='bottom', fontweight='bold')

        ax1.grid(axis='y', alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)

        # نمودار دایره‌ای توزیع شباهت
        similarity_ranges = {
            'بالا (80-100%)': sum(1 for s in similarities if s >= 80),
            'متوسط (60-80%)': sum(1 for s in similarities if 60 <= s < 80),
            'پایین (40-60%)': sum(1 for s in similarities if 40 <= s < 60),
            'خیلی پایین (<40%)': sum(1 for s in similarities if s < 40)
        }

        # حذف دسته‌های خالی
        similarity_ranges = {k: v for k, v in similarity_ranges.items() if v > 0}

        if similarity_ranges:
            colors_pie = ['#2E8B57', '#FFD700', '#FF6347', '#DC143C']
            wedges, texts, autotexts = ax2.pie(
                similarity_ranges.values(),
                labels=similarity_ranges.keys(),
                autopct='%1.1f%%',
                colors=colors_pie[:len(similarity_ranges)],
                startangle=90
            )

            ax2.set_title('🎯 توزیع میزان شباهت', fontsize=14, fontweight='bold')

            # بهبود ظاهر متن‌ها
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')

        plt.tight_layout()
        return fig

    def create_model_comparison_chart(self, query_image, k=5):
        """مقایسه عملکرد مدل‌ها"""
        if not query_image:
            return None

        plt.rcParams['font.family'] = ['DejaVu Sans']

        fig, ax = plt.subplots(figsize=(12, 8))

        model_results = {}
        colors = {'resnet50': '#FF6B6B', 'vit': '#4ECDC4', 'clip': '#45B7D1'}

        for model_type in self.model_types:
            results = self.search_by_image(query_image, model_type, k=k)
            if results:
                similarities = [r['similarity'] * 100 for r in results]
                model_results[model_type.upper()] = similarities

        # نمودار خطی مقایسه
        x_pos = range(1, k + 1)

        for model, sims in model_results.items():
            ax.plot(x_pos[:len(sims)], sims,
                   marker='o', linewidth=3, markersize=8,
                   label=model, color=colors.get(model.lower(), '#333333'))

            # اضافه کردن مقادیر روی نقاط
            for i, sim in enumerate(sims):
                ax.annotate(f'{sim:.1f}%',
                           (i+1, sim),
                           textcoords="offset points",
                           xytext=(0,10),
                           ha='center', fontweight='bold')

        ax.set_title('📈 مقایسه عملکرد مدل‌ها', fontsize=16, fontweight='bold')
        ax.set_xlabel('رتبه نتایج', fontsize=12)
        ax.set_ylabel('درصد شباهت', fontsize=12)
        ax.set_xticks(x_pos)
        ax.set_xticklabels([f'#{i}' for i in x_pos])
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(0, 100)

        plt.tight_layout()
        return fig

    def create_feature_space_visualization(self, query_image, model_type, k=30):
        """نمودار کاهش ابعاد و نمایش فضای ویژگی با t-SNE"""
        if not query_image or model_type not in self.indices:
            return None

        plt.rcParams['font.family'] = ['DejaVu Sans']

        # استخراج ویژگی تصویر پرس‌وجو
        query_features = self.extract_features(query_image, model_type)

        # انتخاب نمونه‌ای از تصاویر دیتابیس
        sample_size = min(k, len(self.image_paths))
        sample_indices = np.random.choice(len(self.image_paths), sample_size, replace=False)

        # ترکیب ویژگی‌های پرس‌وجو با نمونه‌ها
        features_to_plot = [query_features]
        for idx in sample_indices:
            features_to_plot.append(self.features[model_type][idx])

        # تبدیل به آرایه
        features_array = np.array(features_to_plot)

        # کاهش ابعاد با t-SNE
        tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(features_array)-1))
        embedded_features = tsne.fit_transform(features_array)

        # ایجاد نمودار
        fig, ax = plt.subplots(figsize=(10, 8))

        # رنگ‌ها برای نقاط
        colors = ['#FF5733', '#C70039', '#900C3F', '#581845', '#2874A6',
                 '#1ABC9C', '#F1C40F', '#27AE60', '#884EA0', '#D35400']

        # نمایش نقطه پرس‌وجو
        ax.scatter(embedded_features[0, 0], embedded_features[0, 1],
                  s=150, c='red', marker='*', label='تصویر پرس‌وجو', zorder=10)

        # نمایش نقاط دیتابیس
        scatter = ax.scatter(embedded_features[1:, 0], embedded_features[1:, 1],
                            s=80, c=np.arange(len(embedded_features)-1),
                            cmap='viridis', alpha=0.7, label='تصاویر دیتابیس')

        # نمایش خطوط اتصال به نزدیک‌ترین همسایه‌ها
        distances = np.linalg.norm(embedded_features[1:] - embedded_features[0], axis=1)
        closest_indices = np.argsort(distances)[:5] # 5 همسایه نزدیک

        for idx in closest_indices:
            ax.plot([embedded_features[0, 0], embedded_features[idx+1, 0]],
                   [embedded_features[0, 1], embedded_features[idx+1, 1]],
                   'r--', alpha=0.4, linewidth=1)

        # افزودن برچسب‌ها
        ax.set_title(f'نمایش فضای ویژگی با t-SNE ({model_type.upper()})', fontsize=14, fontweight='bold')
        ax.set_xlabel('t-SNE بعد اول', fontsize=12)
        ax.set_ylabel('t-SNE بعد دوم', fontsize=12)

        # افزودن راهنما
        ax.legend(loc='upper right', fontsize=10)

        # افزودن رنگ‌نما
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('شاخص تصویر', fontsize=10)

        plt.tight_layout()
        return fig

    def create_similarity_distribution_analysis(self, results):
        """تحلیل توزیع شباهت نتایج"""
        if not results:
            return None

        plt.rcParams['font.family'] = ['DejaVu Sans']

        similarities = [r['similarity'] for r in results]

        # ایجاد نمودار با دو زیرنمودار
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # 1. نمودار توزیع (هیستوگرام)
        sns.histplot(similarities, bins=10, kde=True, ax=ax1, color='#3498db')
        ax1.set_title('توزیع شباهت نتایج', fontsize=14, fontweight='bold')
        ax1.set_xlabel('میزان شباهت', fontsize=12)
        ax1.set_ylabel('فراوانی', fontsize=12)

        # خط میانگین
        mean_sim = np.mean(similarities)
        ax1.axvline(mean_sim, color='r', linestyle='--', linewidth=2)
        ax1.text(mean_sim, ax1.get_ylim()[1]*0.9, f'میانگین: {mean_sim:.2f}',
                color='r', fontweight='bold', ha='center', backgroundcolor='white')

        # 2. نمودار جعبه‌ای
        sns.boxplot(y=similarities, ax=ax2, color='#2ecc71')
        ax2.set_title('نمودار جعبه‌ای شباهت', fontsize=14, fontweight='bold')
        ax2.set_ylabel('میزان شباهت', fontsize=12)

        # افزودن آمار توصیفی
        stats_text = (
            f"میانگین: {np.mean(similarities):.3f}\n"
            f"میانه: {np.median(similarities):.3f}\n"
            f"انحراف معیار: {np.std(similarities):.3f}\n"
            f"حداقل: {np.min(similarities):.3f}\n"
            f"حداکثر: {np.max(similarities):.3f}"
        )

        ax2.text(0.05, 0.05, stats_text, transform=ax2.transAxes,
                bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'),
                fontsize=10, verticalalignment='bottom')

        plt.tight_layout()
        return fig

    def create_similarity_heatmap(self, query_image, top_k=10):
        """ماتریس حرارتی مقایسه شباهت بین مدل‌ها"""
        if not query_image or len(self.model_types) < 2:
            return None

        plt.rcParams['font.family'] = ['DejaVu Sans']

        # جمع‌آوری نتایج از همه مدل‌ها
        all_results = {}
        for model_type in self.model_types:
            results = self.search_by_image(query_image, model_type, k=top_k)
            if results:
                all_results[model_type] = results

        if not all_results:
            return None

        # ساخت ماتریس شباهت بین مدل‌ها
        model_names = list(all_results.keys())
        similarity_matrix = np.zeros((len(model_names), len(model_names)))

        # محاسبه میزان همپوشانی نتایج بین مدل‌ها
        for i, model1 in enumerate(model_names):
            results1 = all_results[model1]
            paths1 = set(r['path'] for r in results1)

            for j, model2 in enumerate(model_names):
                results2 = all_results[model2]
                paths2 = set(r['path'] for r in results2)

                # محاسبه شاخص جاکارد (نسبت اشتراک به اجتماع)
                intersection = len(paths1.intersection(paths2))
                union = len(paths1.union(paths2))

                if union > 0:
                    similarity_matrix[i, j] = intersection / union
                else:
                    similarity_matrix[i, j] = 0

        # ایجاد نمودار حرارتی
        fig, ax = plt.subplots(figsize=(10, 8))

        # نمایش ماتریس حرارتی
        im = ax.imshow(similarity_matrix, cmap='YlOrRd', vmin=0, vmax=1)

        # تنظیم محورها
        ax.set_xticks(np.arange(len(model_names)))
        ax.set_yticks(np.arange(len(model_names)))
        ax.set_xticklabels([m.upper() for m in model_names])
        ax.set_yticklabels([m.upper() for m in model_names])

        # چرخش برچسب‌های محور افقی
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

        # نمایش مقادیر در هر خانه
        for i in range(len(model_names)):
            for j in range(len(model_names)):
                text = ax.text(j, i, f"{similarity_matrix[i, j]:.2f}",
                              ha="center", va="center", color="black" if similarity_matrix[i, j] < 0.7 else "white",
                              fontweight="bold")

        # عنوان و نوار رنگی
        ax.set_title("ماتریس همپوشانی نتایج بین مدل‌ها", fontsize=14, fontweight='bold')
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label('میزان همپوشانی (شاخص جاکارد)', fontsize=12)

        plt.tight_layout()
        return fig

    def create_confidence_analysis(self, results):
        """تحلیل اطمینان نتایج"""
        if not results or len(results) < 3:
            return None

        plt.rcParams['font.family'] = ['DejaVu Sans']

        similarities = [r['similarity'] for r in results]
        ranks = [r['rank'] for r in results]

        # محاسبه نرخ کاهش شباهت
        decay_rates = []
        for i in range(1, len(similarities)):
            if similarities[i-1] > 0: # جلوگیری از تقسیم بر صفر
                decay = (similarities[i-1] - similarities[i]) / similarities[i-1]
                decay_rates.append(decay)

        # ایجاد نمودار
        fig, ax = plt.subplots(figsize=(12, 7))

        # نمودار خط شباهت
        ax.plot(ranks, similarities, 'o-', color='#3498db', linewidth=2, markersize=8, label='شباهت')

        # ناحیه اطمینان بالا
        high_confidence_threshold = 0.75
        ax.axhspan(high_confidence_threshold, 1.0, alpha=0.2, color='green', label=f'اطمینان بالا (>{high_confidence_threshold:.0%})')

        # ناحیه اطمینان متوسط
        medium_confidence_threshold = 0.5
        ax.axhspan(medium_confidence_threshold, high_confidence_threshold, alpha=0.2, color='yellow', label=f'اطمینان متوسط ({medium_confidence_threshold:.0%}-{high_confidence_threshold:.0%})')

        # ناحیه اطمینان پایین
        ax.axhspan(0, medium_confidence_threshold, alpha=0.2, color='red', label=f'اطمینان پایین (<{medium_confidence_threshold:.0%})')

        # یافتن نقطه شکست (جایی که نرخ کاهش بیش از آستانه است)
        if decay_rates:
            significant_drop_threshold = 0.2 # آستانه افت قابل توجه
            drop_points = [i+2 for i, rate in enumerate(decay_rates) if rate > significant_drop_threshold]

            if drop_points:
                first_drop = drop_points[0]
                ax.axvline(x=first_drop, color='purple', linestyle='--', linewidth=2,
                          label=f'نقطه شکست (رتبه {first_drop})')

                # افزودن متن توضیح
                ax.text(first_drop + 0.1, ax.get_ylim()[1]*0.9,
                       f'افت قابل توجه در شباهت',
                       color='purple', fontweight='bold')

        # تنظیمات نمودار
        ax.set_title('تحلیل اطمینان نتایج جستجو', fontsize=14, fontweight='bold')
        ax.set_xlabel('رتبه', fontsize=12)
        ax.set_ylabel('میزان شباهت', fontsize=12)
        ax.set_ylim(0, 1.05)
        ax.set_xlim(0.5, len(similarities) + 0.5)
        ax.grid(True, alpha=0.3)

        # افزودن برچسب مقادیر روی نقاط
        for i, (x, y) in enumerate(zip(ranks, similarities)):
            ax.annotate(f'{y:.2f}', (x, y), xytext=(0, 5),
                       textcoords='offset points', ha='center', fontsize=8)

        ax.legend(loc='lower left', fontsize=10)

        plt.tight_layout()
        return fig

    def build_indices(self, image_folder):
        """ساخت ایندکس برای همه مدل‌ها"""
        print(f"🔨 ساخت ایندکس‌ها...")

        self.image_paths = []
        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                self.image_paths.append(os.path.join(image_folder, filename))

        print(f"📁 تعداد تصاویر: {len(self.image_paths)}")

        if len(self.image_paths) == 0:
            return False

        for model_type in self.model_types:
            print(f"\n🔧 پردازش با مدل {model_type.upper()}...")

            features_list = []
            for img_path in tqdm(self.image_paths, desc=f"استخراج ویژگی {model_type}"):
                try:
                    features = self.extract_features(img_path, model_type)
                    features_list.append(features)
                except Exception as e:
                    print(f"خطا در پردازش {img_path}: {e}")
                    features_list.append(np.zeros(self.feature_dims[model_type]))

            self.features[model_type] = np.array(features_list).astype('float32')

            self.indices[model_type] = faiss.IndexFlatIP(self.feature_dims[model_type])
            self.indices[model_type].add(self.features[model_type])

            print(f"✅ ایندکس {model_type} با {self.indices[model_type].ntotal} تصویر ساخته شد")

        return True

    def search_by_image(self, query_image, model_type, k=10):
        """جستجو با تصویر"""
        if model_type not in self.indices:
            return []

        query_features = self.extract_features(query_image, model_type)
        query_features = query_features.reshape(1, -1).astype('float32')

        distances, indices = self.indices[model_type].search(query_features, k)

        results = []
        for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
            if idx < len(self.image_paths):
                original_img = Image.open(self.image_paths[idx])
                img_with_score = self.add_similarity_text_to_image(original_img, dist, rank)

                results.append({
                    'path': self.image_paths[idx],
                    'similarity': float(dist),
                    'model': model_type,
                    'rank': rank,
                    'image': img_with_score,
                    'original_image': original_img
                })

        return results

    def search_by_text(self, text_query, k=10):
        """جستجو با متن (فقط CLIP)"""
        if 'clip' not in self.indices:
            return []

        inputs = self.processors['clip'](text=[text_query], return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            text_features = self.models['clip'].get_text_features(**inputs)
            text_features = text_features.squeeze().cpu().numpy()
            text_features = text_features / np.linalg.norm(text_features)

        text_features = text_features.reshape(1, -1).astype('float32')
        distances, indices = self.indices['clip'].search(text_features, k)

        results = []
        for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
            if idx < len(self.image_paths):
                original_img = Image.open(self.image_paths[idx])
                img_with_score = self.add_similarity_text_to_image(original_img, dist, rank)

                results.append({
                    'path': self.image_paths[idx],
                    'similarity': float(dist),
                    'model': 'clip',
                    'rank': rank,
                    'image': img_with_score,
                    'original_image': original_img
                })

        return results

    def get_sample_images(self, count=6):
        """دریافت نمونه تصاویر از دیتابیس"""
        if len(self.image_paths) == 0:
            return []

        sample_paths = random.sample(self.image_paths, min(count, len(self.image_paths)))
        return sample_paths

class CarImageDownloader:
    """دانلود تصاویر متنوع شامل خودرو"""

    def __init__(self, base_dir='/content/image_database'):
        self.base_dir = base_dir
        os.makedirs(base_dir, exist_ok=True)

    def download_diverse_images(self, count=60):
        """دانلود تصاویر متنوع شامل خودرو"""
        print(f"📥 دانلود {count} تصویر متنوع شامل خودرو...")

        downloaded = 0

        for i in tqdm(range(count), desc="دانلود"):
            try:
                url = f"https://picsum.photos/400/400?random={i+2000}"

                response = requests.get(url, timeout=10)
                if response.status_code == 200:
                    img = Image.open(BytesIO(response.content))
                    img = img.convert('RGB')

                    filename = f"img_{i:03d}.jpg"
                    filepath = os.path.join(self.base_dir, filename)
                    img.save(filepath)
                    downloaded += 1

                time.sleep(0.1)

            except Exception as e:
                print(f"خطا در دانلود تصویر {i}: {e}")
                continue

        print(f"✅ {downloaded} تصویر دانلود شد")
        return downloaded

# متغیر سراسری
search_engine = None

def initialize_system():
    """راه‌اندازی سیستم"""
    global search_engine

    downloader = CarImageDownloader()
    downloaded_count = downloader.download_diverse_images(count=60)

    if downloaded_count == 0:
        return False, "خطا در دانلود تصاویر"

    search_engine = MultiModalImageSearchEngine(['resnet50', 'vit', 'clip'])

    success = search_engine.build_indices('/content/image_database')

    if success:
        return True, f"سیستم با {downloaded_count} تصویر و 3 مدل آماده است"
    else:
        return False, "خطا در ساخت ایندکس"

# اجرای سیستم
print("🚀 راه‌اندازی موتور جستجوی تصویری چند مدله...")

# راه‌اندازی
success, message = initialize_system()
print(f"📊 {message}")

if success:
    print("🎨 راه‌اندازی رابط کاربری Gradio...")

    # دریافت نمونه تصاویر
    sample_paths = search_engine.get_sample_images(6)
    sample_images = []
    for path in sample_paths:
        try:
            img = Image.open(path)
            sample_images.append(img)
        except:
            continue

    def search_images_gradio(query_image, query_text, search_mode, model_type, num_results):
        global search_engine

        if search_engine is None:
            return [], "", None, None, None, None, None

        try:
            if search_mode == "جستجو با تصویر":
                if query_image is None:
                    return [], "لطفاً تصویر آپلود کنید", None, None, None, None, None
                results = search_engine.search_by_image(query_image, model_type, k=num_results)
                query_type = "تصویری"
            else:
                if not query_text:
                    return [], "لطفاً متن وارد کنید", None, None, None, None, None
                if model_type != 'clip':
                    return [], "جستجوی متنی فقط با CLIP امکان‌پذیر است", None, None, None, None, None
                results = search_engine.search_by_text(query_text, k=num_results)
                query_type = "متنی"

            if not results:
                return [], "نتیجه‌ای یافت نشد", None, None, None, None, None

            # تصاویر با نمایش شباهت
            output_images = []
            info_text = f"🔍 {len(results)} نتیجه با مدل {model_type.upper()}:\n\n"

            for result in results:
                img_with_score = result['image']
                output_images.append(img_with_score)
                info_text += f"#{result['rank']}. شباهت: {result['similarity']:.1%}\n"

            # ایجاد نمودارهای تحلیلی
            similarity_chart = search_engine.create_similarity_chart(results, query_type)

            # نمودار مقایسه مدل‌ها (فقط برای جستجوی تصویری)
            comparison_chart = None
            if search_mode == "جستجو با تصویر" and query_image is not None:
                comparison_chart = search_engine.create_model_comparison_chart(query_image, k=5)

            # نمودارهای جدید
            feature_space_viz = None
            similarity_dist = None
            confidence_analysis = None

            if search_mode == "جستجو با تصویر" and query_image is not None:
                # نمایش فضای ویژگی
                feature_space_viz = search_engine.create_feature_space_visualization(query_image, model_type)

            # تحلیل توزیع شباهت برای همه انواع جستجو
            similarity_dist = search_engine.create_similarity_distribution_analysis(results)

            # تحلیل اطمینان نتایج
            confidence_analysis = search_engine.create_confidence_analysis(results)

            return (output_images, info_text, similarity_chart, comparison_chart,
                    feature_space_viz, similarity_dist, confidence_analysis)

        except Exception as e:
            return [], f"خطا: {str(e)}", None, None, None, None, None

    with gr.Blocks(title="موتور جستجوی تصویری چند مدله", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🔍 موتور جستجوی تصویری چند مدله با تحلیل پیشرفته

        ### مدل‌های پشتیبانی شده:
        - **ResNet50** (CNN): شبکه عصبی کانولوشنال
        - **ViT** (Vision Transformer): ترنسفورمر بینایی
        - **CLIP**: مدل چندوجهی متن-تصویر
        """)

        with gr.Row():
            with gr.Column(scale=1):
                search_mode = gr.Radio(
                    ["جستجو با تصویر", "جستجو با متن"],
                    value="جستجو با تصویر",
                    label="نوع جستجو"
                )

                model_type = gr.Dropdown(
                    choices=["resnet50", "vit", "clip"],
                    value="clip",
                    label="انتخاب مدل"
                )

                query_image = gr.Image(type="pil", label="تصویر ورودی")
                query_text = gr.Textbox(
                    label="متن جستجو",
                    placeholder="مثال: red car, beautiful landscape, modern building",
                    visible=False
                )

                num_results = gr.Slider(1, 20, value=12, step=1, label="تعداد نتایج")
                search_btn = gr.Button("🔍 جستجو", variant="primary")

            with gr.Column(scale=2):
                output_gallery = gr.Gallery(
                    label="نتایج جستجو (با نمایش میزان شباهت)",
                    columns=3,
                    rows=4,
                    height="auto"
                )

                info_text = gr.Textbox(label="اطلاعات نتایج", lines=8)

                # نمودارهای تحلیلی - تب‌بندی
                with gr.Tabs():
                    with gr.TabItem("📊 نمودار شباهت"):
                        similarity_plot = gr.Plot(label="نمودار شباهت")

                    with gr.TabItem("📈 مقایسه مدل‌ها"):
                        comparison_plot = gr.Plot(label="مقایسه مدل‌ها")

                    with gr.TabItem("🔬 فضای ویژگی"):
                        feature_space_plot = gr.Plot(label="نمایش t-SNE فضای ویژگی")

                    with gr.TabItem("📉 تحلیل توزیع"):
                        similarity_dist_plot = gr.Plot(label="تحلیل توزیع شباهت")

                    with gr.TabItem("🎯 تحلیل اطمینان"):
                        confidence_plot = gr.Plot(label="تحلیل اطمینان نتایج")

        # تغییر نمایش
        def update_visibility(mode):
            if mode == "جستجو با تصویر":
                return gr.update(visible=True), gr.update(visible=False)
            else:
                return gr.update(visible=False), gr.update(visible=True)

        search_mode.change(
            fn=update_visibility,
            inputs=[search_mode],
            outputs=[query_image, query_text]
        )

        # اتصال جستجو
        search_btn.click(
            fn=search_images_gradio,
            inputs=[query_image, query_text, search_mode, model_type, num_results],
            outputs=[output_gallery, info_text, similarity_plot, comparison_plot,
                    feature_space_plot, similarity_dist_plot, confidence_plot]
        )

        # نمونه‌های جستجو
        gr.Markdown("### 📝 نمونه‌های جستجو برای همه مدل‌ها:")

        # ایجاد نمونه‌ها با تصاویر واقعی از دیتابیس
        examples_list = []

        # نمونه‌های متنی (فقط برای CLIP)
        examples_list.extend([
            [None, "red sports car", "جستجو با متن", "clip", 8],
            [None, "beautiful sunset", "جستجو با متن", "clip", 8],
            [None, "mountain landscape", "جستجو با متن", "clip", 8],
        ])

        # نمونه‌های تصویری برای همه مدل‌ها
        if len(sample_images) >= 3:
            # ResNet50
            examples_list.append([sample_images[0], "", "جستجو با تصویر", "resnet50", 8])
            # ViT
            examples_list.append([sample_images[1], "", "جستجو با تصویر", "vit", 8])
            # CLIP
            examples_list.append([sample_images[2], "", "جستجو با تصویر", "clip", 8])

        gr.Examples(
            examples=examples_list,
            inputs=[query_image, query_text, search_mode, model_type, num_results],
            label="کلیک کنید تا جستجو شود"
        )

        gr.Markdown("""
        ### 💡 راهنمای تحلیل‌های پیشرفته:

        - **نمودار شباهت**: توزیع و میزان شباهت نتایج
        - **مقایسه مدل‌ها**: عملکرد مدل‌های مختلف (فقط جستجوی تصویری)
        - **فضای ویژگی**: نمایش t-SNE از فضای ویژگی و همسایه‌های نزدیک
        - **تحلیل توزیع**: توزیع آماری شباهت‌ها با هیستوگرام و نمودار جعبه‌ای
        - **تحلیل اطمینان**: بررسی نقاط شکست در شباهت و سطوح اطمینان
        """)

    demo.launch(share=True, debug=True)
    print("✅ سیستم کامل با نمودارهای تحلیلی پیشرفته آماده است!")
    print("📊 نمودارهای شباهت، مقایسه مدل‌ها، فضای ویژگی، توزیع شباهت و تحلیل اطمینان اضافه شده‌اند")
else:
    print("❌ خطا در راه‌اندازی")

🚀 راه‌اندازی موتور جستجوی تصویری چند مدله...
📥 دانلود 60 تصویر متنوع شامل خودرو...


دانلود: 100%|██████████| 60/60 [00:38<00:00,  1.56it/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ 60 تصویر دانلود شد
🚀 بارگذاری مدل‌ها روی cuda...
📦 بارگذاری ResNet50...
📦 بارگذاری ViT...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📦 بارگذاری CLIP...
✅ همه مدل‌ها بارگذاری شدند!
🔨 ساخت ایندکس‌ها...
📁 تعداد تصاویر: 60

🔧 پردازش با مدل RESNET50...


...

استخراج ویژگی clip: 100%|██████████| 60/60 [00:00<00:00, 80.97it/s]


✅ ایندکس clip با 60 تصویر ساخته شد
📊 سیستم با 60 تصویر و 3 مدل آماده است
🎨 راه‌اندازی رابط کاربری Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30154b2493f2aeafa8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-4-2835459202.py:195: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-4-2835459202.py:195: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipython-input-4-2835459202.py:241: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:158: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:158: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, format=fmt)
/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:158: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(output_bytes, forma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://43e49183ea598f1e29.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://991c85f6ffd4fc1071.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://30154b2493f2aeafa8.gradio.live
✅ سیستم کامل با نمودارهای تحلیلی پیشرفته آماده است!
📊 نمودارهای شباهت، مقایسه مدل‌ها، فضای ویژگی، توزیع شباهت و تحلیل اطمینان اضافه شده‌اند


/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  func(*args, **kwargs)


# **BASED ON GOOGLE IMAGE**

In [2]:
# نصب کتابخانه‌های مورد نیاز
!pip install -q faiss-cpu torch torchvision transformers gradio pillow requests tqdm

import os
import torch
import numpy as np
import faiss
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from tqdm import tqdm
import time
import random

# Import models
from torchvision import models, transforms
from transformers import CLIPModel, CLIPProcessor, ViTModel, ViTImageProcessor

class ImageSearchEngine:
    """موتور جستجوی تصویری ساده"""

    def __init__(self, model_types=['resnet50', 'vit', 'clip']):
        self.model_types = model_types
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = {}
        self.processors = {}
        self.transforms = {}
        self.feature_dims = {}
        self.indices = {}
        self.image_paths = []
        self.features = {}

        print(f"🚀 بارگذاری مدل‌ها روی {self.device}...")
        self._load_all_models()

    def _load_all_models(self):
        """بارگذاری همه مدل‌ها"""

        # 1. ResNet50 (CNN)
        if 'resnet50' in self.model_types:
            print("📦 بارگذاری ResNet50...")
            self.models['resnet50'] = models.resnet50(pretrained=True)
            self.models['resnet50'] = torch.nn.Sequential(*list(self.models['resnet50'].children())[:-1])
            self.models['resnet50'].to(self.device)
            self.models['resnet50'].eval()
            self.feature_dims['resnet50'] = 2048

            self.transforms['resnet50'] = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
            ])

        # 2. Vision Transformer (ViT)
        if 'vit' in self.model_types:
            print("📦 بارگذاری ViT...")
            self.processors['vit'] = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
            self.models['vit'] = ViTModel.from_pretrained('google/vit-base-patch16-224')
            self.models['vit'].to(self.device)
            self.models['vit'].eval()
            self.feature_dims['vit'] = 768

        # 3. CLIP
        if 'clip' in self.model_types:
            print("📦 بارگذاری CLIP...")
            self.models['clip'] = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
            self.processors['clip'] = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
            self.models['clip'].to(self.device)
            self.models['clip'].eval()
            self.feature_dims['clip'] = 512

        print("✅ همه مدل‌ها بارگذاری شدند!")

    def extract_features(self, image, model_type):
        """استخراج ویژگی با مدل مشخص"""
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')

        with torch.no_grad():
            if model_type == 'resnet50':
                img_tensor = self.transforms['resnet50'](image).unsqueeze(0).to(self.device)
                features = self.models['resnet50'](img_tensor).squeeze().cpu().numpy()

            elif model_type == 'vit':
                inputs = self.processors['vit'](images=image, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                outputs = self.models['vit'](**inputs)
                features = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

            elif model_type == 'clip':
                inputs = self.processors['clip'](images=image, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                features = self.models['clip'].get_image_features(**inputs).squeeze().cpu().numpy()

            return features / np.linalg.norm(features) # نرمال‌سازی

    def add_similarity_badge(self, image, similarity_score, rank=None):
        """اضافه کردن نشان شباهت به تصویر (شبیه گوگل)"""
        img_copy = image.copy()
        draw = ImageDraw.Draw(img_copy)

        try:
            font = ImageFont.load_default()
        except:
            font = ImageFont.load_default()

        badge_text = f"{similarity_score:.0%}"

        # موقعیت نشان - گوشه بالا سمت چپ
        x, y = 10, 10

        # کشیدن مستطیل نیمه شفاف
        draw.rectangle([x, y, x + 50, y + 20], fill=(0, 0, 0, 160))

        # متن وسط مستطیل
        draw.text((x + 5, y + 5), badge_text, fill=(255, 255, 255), font=font)

        return img_copy

    def build_indices(self, image_folder):
        """ساخت ایندکس برای همه مدل‌ها"""
        print(f"🔨 ساخت ایندکس‌ها...")

        self.image_paths = []
        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                self.image_paths.append(os.path.join(image_folder, filename))

        print(f"📁 تعداد تصاویر: {len(self.image_paths)}")

        if len(self.image_paths) == 0:
            return False

        for model_type in self.model_types:
            print(f"\n🔧 پردازش با مدل {model_type.upper()}...")

            features_list = []
            for img_path in tqdm(self.image_paths, desc=f"استخراج ویژگی {model_type}"):
                try:
                    features = self.extract_features(img_path, model_type)
                    features_list.append(features)
                except Exception as e:
                    print(f"خطا در پردازش {img_path}: {e}")
                    features_list.append(np.zeros(self.feature_dims[model_type]))

            self.features[model_type] = np.array(features_list).astype('float32')

            self.indices[model_type] = faiss.IndexFlatIP(self.feature_dims[model_type])
            self.indices[model_type].add(self.features[model_type])

            print(f"✅ ایندکس {model_type} با {self.indices[model_type].ntotal} تصویر ساخته شد")

        return True

    def search_by_image(self, query_image, model_type, k=10):
        """جستجو با تصویر"""
        if model_type not in self.indices:
            return []

        query_features = self.extract_features(query_image, model_type)
        query_features = query_features.reshape(1, -1).astype('float32')

        distances, indices = self.indices[model_type].search(query_features, k)

        results = []
        for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
            if idx < len(self.image_paths):
                original_img = Image.open(self.image_paths[idx])
                img_with_score = self.add_similarity_badge(original_img, dist, rank)

                results.append({
                    'path': self.image_paths[idx],
                    'similarity': float(dist),
                    'model': model_type,
                    'rank': rank,
                    'image': img_with_score
                })

        return results

    def search_by_text(self, text_query, k=10):
        """جستجو با متن (فقط CLIP)"""
        if 'clip' not in self.indices:
            return []

        inputs = self.processors['clip'](text=[text_query], return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            text_features = self.models['clip'].get_text_features(**inputs)
            text_features = text_features.squeeze().cpu().numpy()
            text_features = text_features / np.linalg.norm(text_features)

            text_features = text_features.reshape(1, -1).astype('float32')
            distances, indices = self.indices['clip'].search(text_features, k)

            results = []
            for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), 1):
                if idx < len(self.image_paths):
                    original_img = Image.open(self.image_paths[idx])
                    img_with_score = self.add_similarity_badge(original_img, dist, rank)

                    results.append({
                        'path': self.image_paths[idx],
                        'similarity': float(dist),
                        'model': 'clip',
                        'rank': rank,
                        'image': img_with_score
                    })

            return results

    def get_sample_images(self, count=6):
        """دریافت نمونه تصاویر از دیتابیس"""
        if len(self.image_paths) == 0:
            return []

        sample_paths = random.sample(self.image_paths, min(count, len(self.image_paths)))
        return sample_paths

def download_images(base_dir='/content/image_database', count=100):
    """دانلود تصاویر متنوع"""
    os.makedirs(base_dir, exist_ok=True)
    print(f"📥 دانلود {count} تصویر متنوع...")

    downloaded = 0

    # دانلود از Picsum Photos
    for i in tqdm(range(count), desc="دانلود"):
        try:
            url = f"https://picsum.photos/500/400?random={i+2000}"

            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img = img.convert('RGB')

                filename = f"img_{i:03d}.jpg"
                filepath = os.path.join(base_dir, filename)
                img.save(filepath)
                downloaded += 1

                time.sleep(0.1)

        except Exception as e:
            print(f"خطا در دانلود تصویر {i}: {e}")
            continue

    print(f"✅ {downloaded} تصویر دانلود شد")
    return downloaded

# راه‌اندازی سیستم
print("🚀 راه‌اندازی موتور جستجوی تصویری شبیه گوگل...")

# دانلود تصاویر
downloaded_count = download_images(count=50)

if downloaded_count > 0:
    # ایجاد موتور جستجو
    search_engine = ImageSearchEngine(['resnet50', 'vit', 'clip'])

    # ساخت ایندکس‌ها
    success = search_engine.build_indices('/content/image_database')

    if success:
        print("🎨 راه‌اندازی رابط کاربری شبیه گوگل...")

        # دریافت نمونه تصاویر
        sample_paths = search_engine.get_sample_images(4)
        sample_images = []
        for path in sample_paths:
            try:
                img = Image.open(path)
                sample_images.append(img)
            except:
                continue

        # تابع جستجو
        def search_images(query_image, query_text, search_mode, model_type, num_results):
            try:
                if search_mode == "تصویر":
                    if query_image is None:
                        return [], "لطفاً تصویر آپلود کنید"
                    results = search_engine.search_by_image(query_image, model_type, k=num_results)
                else:
                    if not query_text:
                        return [], "لطفاً متن جستجو را وارد کنید"
                    if model_type != 'clip':
                        return [], "جستجوی متنی فقط با CLIP امکان‌پذیر است"
                    results = search_engine.search_by_text(query_text, k=num_results)

                if not results:
                    return [], "نتیجه‌ای یافت نشد"

                # آماده‌سازی خروجی
                output_images = [r['image'] for r in results]

                # متن نتایج
                info_text = f"حدود {len(results)} نتیجه با {model_type.upper()} | "
                info_text += f"جستجوی {'تصویری' if search_mode == 'تصویر' else 'متنی'}"

                return output_images, info_text

            except Exception as e:
                return [], f"خطا: {str(e)}"

        # رابط کاربری
        with gr.Blocks(title="جستجوی تصویری شبیه گوگل") as demo:
            gr.HTML("""
            <div style="display: flex; justify-content: center; margin-bottom: 20px;">
                <div style="font-size: 22px; font-weight: 500;">
                    <span style="color: #4285f4;">I</span><span style="color: #ea4335;">m</span><span style="color: #fbbc05;">a</span><span style="color: #4285f4;">g</span><span style="color: #34a853;">e</span>
                    <span style="margin-left: 5px;">Search</span>
                </div>
            </div>
            """)

            with gr.Row():
                search_mode = gr.Radio(
                    ["متن", "تصویر"],
                    value="متن",
                    label="نوع جستجو"
                )

            with gr.Row():
                query_text = gr.Textbox(
                    label="",
                    placeholder="جستجوی تصاویر...",
                    container=False
                )

                query_image = gr.Image(
                    label="",
                    type="pil",
                    visible=False
                )

            with gr.Row():
                search_btn = gr.Button("🔍 جستجو")

            with gr.Accordion("تنظیمات", open=False):
                model_type = gr.Dropdown(
                    choices=["resnet50", "vit", "clip"],
                    value="clip",
                    label="انتخاب مدل"
                )

                num_results = gr.Slider(
                    minimum=5,
                    maximum=30,
                    value=12,
                    step=1,
                    label="تعداد نتایج"
                )

            # نمایش متن نتایج
            result_info = gr.Textbox(
                label="",
                lines=1
            )

            # گرید نتایج
            result_gallery = gr.Gallery(
                label="نتایج جستجو",
                columns=4,
                rows=3,
                height="auto",
                object_fit="cover"
            )

            # تغییر نمایش براساس نوع جستجو
            def update_search_mode(mode):
                if mode == "متن":
                    return gr.update(visible=True), gr.update(visible=False)
                else:
                    return gr.update(visible=False), gr.update(visible=True)

            search_mode.change(
                fn=update_search_mode,
                inputs=[search_mode],
                outputs=[query_text, query_image]
            )

            # اتصال جستجو
            search_btn.click(
                fn=search_images,
                inputs=[query_image, query_text, search_mode, model_type, num_results],
                outputs=[result_gallery, result_info]
            )

            # نمونه‌های جستجو
            gr.Markdown("### نمونه‌های جستجو:")

            # نمونه‌های متنی
            text_examples = [
                "red sports car",
                "beautiful landscape with mountains",
                "modern architecture building",
                "sunset over ocean",
                "luxury vehicle"
            ]

            gr.Examples(
                examples=[[ex] for ex in text_examples],
                inputs=[query_text],
                label="نمونه‌های متنی"
            )

            # نمونه‌های تصویری
            if len(sample_images) > 0:
                gr.Examples(
                    examples=[[img] for img in sample_images],
                    inputs=[query_image],
                    label="نمونه‌های تصویری"
                )

        demo.launch(share=True)
        print("✅ موتور جستجوی تصویری با رابط کاربری شبیه گوگل راه‌اندازی شد!")
    else:
        print("❌ خطا در ساخت ایندکس")
else:
    print("❌ خطا در دانلود تصاویر")

🚀 راه‌اندازی موتور جستجوی تصویری شبیه گوگل...
📥 دانلود 50 تصویر متنوع...


دانلود: 100%|██████████| 50/50 [00:48<00:00,  1.04it/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ 46 تصویر دانلود شد
🚀 بارگذاری مدل‌ها روی cuda...
📦 بارگذاری ResNet50...
📦 بارگذاری ViT...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📦 بارگذاری CLIP...
✅ همه مدل‌ها بارگذاری شدند!
🔨 ساخت ایندکس‌ها...
📁 تعداد تصاویر: 50

🔧 پردازش با مدل RESNET50...


...

استخراج ویژگی vit: 100%|██████████| 50/50 [00:00<00:00, 55.94it/s]


✅ ایندکس vit با 50 تصویر ساخته شد

🔧 پردازش با مدل CLIP...


استخراج ویژگی clip: 100%|██████████| 50/50 [00:00<00:00, 76.24it/s]


✅ ایندکس clip با 50 تصویر ساخته شد
🎨 راه‌اندازی رابط کاربری شبیه گوگل...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://991c85f6ffd4fc1071.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ موتور جستجوی تصویری با رابط کاربری شبیه گوگل راه‌اندازی شد!
